In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from IPython.display import display
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import acf, adfuller, pacf

# Load & preprocessing the data

## Barley

In [ ]:
barley_yield = pd.read_csv("data/barley_yield_from_1982.csv", sep=";")
barley_yield

In [ ]:
barley_yield

## climate data

In [ ]:
climate_data = pd.read_parquet("data/climate_data_from_1982.parquet")
climate_data

In [ ]:
print("Memory before optimization (MB):")
print(climate_data.memory_usage(deep=True).sum() / 1e6)
# Convert categorical columns
optimized_df = climate_data.copy()
categorical_cols = ["scenario", "nom_dep", "code_dep", "metric"]
for col in categorical_cols:
    optimized_df[col] = optimized_df[col].astype("category")

# Convert year to int16
optimized_df["year"] = optimized_df["year"].astype("int16")

print("Memory after optimization (MB):")
print(optimized_df.memory_usage(deep=True).sum() / 1e6)

In [ ]:
optimized_df.dtypes

In [ ]:
# extracting metrics
metric_df = optimized_df.pivot(
    index=["scenario", "nom_dep", "code_dep", "time", "year"], columns="metric", values="value"
).reset_index()
metric_df.columns.name = None
metric_df

## creating dfs based on scenarios

In [ ]:
ssp5_8_5 = metric_df[metric_df["scenario"] == "ssp5_8_5"]
ssp2_4_5 = metric_df[metric_df["scenario"] == "ssp2_4_5"]
ssp1_2_6 = metric_df[metric_df["scenario"] == "ssp1_2_6"]
historical = metric_df[metric_df["scenario"] == "historical"]

In [ ]:
datasets_names = ["historical", "ssp1_2_6", "ssp2_4_5", "ssp5_8_5"]
datasets = [historical, ssp1_2_6, ssp2_4_5, ssp5_8_5]

In [ ]:
ssp2_4_5.isna().sum()

# General descriptive info climate_data

In [ ]:
for col in optimized_df.columns:
    print(col, "\n", climate_data[col].unique())

In [ ]:
optimized_df.shape

In [ ]:
optimized_df.info()

In [ ]:
optimized_df.isna().sum()

In [ ]:
optimized_df.groupby("metric")["value"].describe()

In [ ]:
yearly_mean = optimized_df.groupby(["year", "metric"])["value"].mean().reset_index()

plt.figure(figsize=(8, 5))
sns.lineplot(data=yearly_mean, x="year", y="value", hue="metric")
plt.title("France Annual Mean by Metric")
plt.show()

# Scenario comparison

In [ ]:
for dataset, name in zip(datasets, datasets_names, strict=True):
    print("\n", f"***************************************{name}*********************************")
    display(dataset.describe())

In [ ]:
temp_df = optimized_df[optimized_df["metric"] == "near_surface_air_temperature"]

scenario_trend = temp_df.groupby(["year", "scenario"])["value"].mean().reset_index()

plt.figure(figsize=(12, 6))
sns.lineplot(data=scenario_trend, x="year", y="value", hue="scenario")
plt.title("Temperature Evolution by Scenario")
plt.show()

In [ ]:
sns.boxplot(data=metric_df, x="scenario", y="near_surface_air_temperature")
plt.title("Mean Temperature Distribution by Scenario")
plt.show()

sns.boxplot(data=metric_df, x="scenario", y="precipitation")
plt.title("Precipitation Distribution by Scenario")
plt.show()

In [ ]:
hot_days = metric_df[metric_df["daily_maximum_near_surface_air_temperature"] > 310]

hot_days_count = hot_days.groupby(["scenario", "year"]).size().reset_index(name="nb_hot_days")

plt.figure(figsize=(12, 6))
sns.lineplot(data=hot_days_count, x="year", y="nb_hot_days", hue="scenario")
plt.title("Number of Hot Days (>310)")
plt.show()

In [ ]:
annual_precip = metric_df.groupby(["scenario", "year"])["precipitation"].sum().reset_index()

sns.lineplot(data=annual_precip, x="year", y="precipitation", hue="scenario")
plt.title("Annual Total Precipitation")
plt.show()

In [ ]:
def plot_hottest_and_coldest_per_scenario(scenario, scenario_name, temperature):
    """temperature: hot or cold
    scenario: dataset of the specific scenario
    scenario_name: dataset name of the specific scenario (str)"""
    if temperature == "cold":
        ascending = True
    if temperature == "hot":
        ascending = False
    if temperature not in ["cold", "hot"]:
        return "Please choose the correct argument for temperature: hot / cold"

    dep_summary = scenario.groupby(["nom_dep"])["near_surface_air_temperature"].mean().reset_index()

    # 10 hottest per scenario
    top10 = dep_summary.sort_values(["near_surface_air_temperature"], ascending=[ascending]).head(
        10
    )

    top10["nom_dep"] = top10["nom_dep"].cat.remove_unused_categories()

    plt.figure(figsize=(12, 6))

    sns.barplot(
        data=top10,
        x="nom_dep",
        y="near_surface_air_temperature",
    )

    plt.xticks(rotation=45)
    if temperature == "cold":
        plt.title(f"10 coldest Departments for {scenario_name} data")
    if temperature == "hot":
        plt.title(f"10 Hottest Departments for {scenario_name} data")
    plt.tight_layout()
    plt.show()


for dataset, name in zip(datasets, datasets_names, strict=True):
    for temperature in ["hot", "cold"]:
        plot_hottest_and_coldest_per_scenario(dataset, name, temperature)

In [ ]:
sns.scatterplot(
    data=metric_df, x="near_surface_air_temperature", y="precipitation", hue="scenario", alpha=0.3
)
plt.show()
plt.title("Temperature vs Precipitation Relationship")

# Variability analysis in each scenario

In [ ]:
from scipy.stats import linregress


def compute_trend(data):
    slope, _, _, _, _ = linregress(data["year"], data["near_surface_air_temperature"])
    return slope


trend = metric_df.groupby("scenario").apply(compute_trend)
print("Linear trend (slope) of near-surface air temperature versus year for each scenario")
print(trend)
print("Each value represents average temperature change per year")
print(
    "All slopes are positive. This means temperature is increasing over time in every scenario.\
    Here we see the slowest warming to the strongest scenario."
)

print("\nVariability changes (standard deviation):")
metric_df.groupby("scenario")["near_surface_air_temperature"].std()

In [ ]:
print(
    "Extreme temperatures (e.g., 99th percentile) increase faster than the mean. \
The hottest days are warming disproportionately compared to average days. \
This implies Increased variability and More extreme heat events"
)

metric_df.groupby("scenario")["daily_maximum_near_surface_air_temperature"].quantile([0.9, 0.99])

In [ ]:
print("Skewness for daily max temperature and for surface air temperature:")
display(metric_df.groupby("scenario")["daily_maximum_near_surface_air_temperature"].skew())

print("\n")
display(metric_df.groupby("scenario")["near_surface_air_temperature"].skew())

print("Climate change amplifies hot extremes more than it shifts average daily temperature.")

In [ ]:
sns.kdeplot(
    data=metric_df,
    x="daily_maximum_near_surface_air_temperature",
    hue="scenario",
    fill=True,
    common_norm=False,
)

plt.title("Distribution of Daily Max Temperature")
plt.show()

In [ ]:
sns.kdeplot(
    data=metric_df, x="near_surface_air_temperature", hue="scenario", fill=True, common_norm=False
)

plt.title("Distribution of Temperature")
plt.show()

# Correlation analysis

In [ ]:
metric_df.groupby("scenario")[
    ["near_surface_air_temperature", "daily_maximum_near_surface_air_temperature", "precipitation"]
].corr()

# Autocorrelation / lag analysis

In [ ]:
series = barley_yield.iloc[:, 1:].sort_values("year")["production"]

fig, ax = plt.subplots(2, 1, figsize=(8, 6))

plot_acf(series, ax=ax[0])
plot_pacf(series, ax=ax[1])

plt.tight_layout()
plt.show()

In [ ]:
barley_yield.columns

In [ ]:
# Plot the ACF and PACF
plt.subplot(211)
plt.plot(acf(barley_yield["production"]))
plt.axhline(0, color="black")
plt.axhline(1.96 / np.sqrt(len(barley_yield)), color="red", linestyle="dashed")
plt.axhline(-1.96 / np.sqrt(len(barley_yield)), color="red", linestyle="dashed")
plt.title("ACF")

plt.subplot(212)
plt.plot(pacf(barley_yield["production"]))
plt.axhline(0, color="black")
plt.axhline(1.96 / np.sqrt(len(barley_yield)), color="red", linestyle="dashed")
plt.axhline(-1.96 / np.sqrt(len(barley_yield)), color="red", linestyle="dashed")
plt.title("PACF")

plt.show()

In [ ]:
adf_result = adfuller(series.dropna())
print("ADF p-value:", adf_result[1])
# (?)

# Transformations / External data to add

- inflation? / cost indices per matiere primaire
- derived data (check for collinearity though):

          - compute number of hot days (threshold) per year,
          - total duration per heatwave (since will delete time details, can include an aggregate per year) as well as total precipitation during season,
          - consecutive dry days
- any weather data (soil ph, or any in agricultural sources such as ISRIC SoilGrids, European Soil Database, FAO soil maps, National agricultural institutes)
- lag features or decay
- feature transformation:
  
          - Normalize per area (eg divide precipitation per area or so and keep that)

- finir EDA
- set up la pipeline

      -clean la data (pivoter (fait), separer les test split sur les predictions (date threshold), enlever les 8 departements dans barley qui ne sont pas present dnas le 245
      -faire les features (
- model dvp